# Graph drawing algorithm A (3 nodes)

In [1]:
import numpy as np
import cvxpy as cp

## Base matrices of the optimization problem
These matrices are written in the real coordinate system. The vertical locations along axis _y_ are stored in vector $\mathbf{Y}$:
$$
\mathbf{Y} = \left< y_{(2,1)} y_{(2,2)}\right>
$$

**Cost function** $f(\mathbf{Y})$:

$$
f(\mathbf{Y}) = \mathbf{c}\cdot\mathbf{Y}
$$

In [2]:
lform_c = np.array([-1, 1])
print("lform_c = \n", lform_c)

lform_c = 
 [-1  1]


**Barycentric** relations between parent and child nodes:
$$
[A]\cdot\mathbf{Y} = 0
$$

In [3]:
mat_a = np.array([[1, 1]])
print("mat_a = \n", mat_a)

mat_a = 
 [[1 1]]


**Minimal distance** relations:
$$
[G]\cdot \mathbf{Y}\ge d_\mathrm{min}
$$

In [4]:
mat_g = np.array([[-1,1]])
print("mat_g = \n", mat_g)

mat_g = 
 [[-1  1]]


**Transformation** to unknown vector $\mathbf{X}$ such that $\mathbf{X} \ge 0$ implies $[G]\cdot \mathbf{Y}\ge d_\mathrm{min}$.

$$
\mathbf{X} = \left<\begin{matrix}
y_{(2,1)} & y_{(2,2)}-y_{(2,1)}-d_\mathrm{min} \end{matrix}\right>
$$
which writes:
$$
\mathbf{X} = [P]\cdot\mathbf{Y}-\mathbf{b}_X
$$
The inverse relationship being:
$$
\mathbf{Y} = [P]^{-1}\cdot\left(\mathbf{X}+\mathbf{b}_X\right)
$$
Remark: matrix $[P]$ is a unipotent [matrix](https://en.wikipedia.org/wiki/Unipotent) matrix.

In [5]:
mat_p = np.array([
    [1, 0],
    [-1, 1]
])
d_min = 30
vect_b_x  = np.array([0, d_min])
inv_mat_p = np.linalg.inv(mat_p)
print("mat_p = \n", mat_p)
print("inv_mat_p = \n", inv_mat_p)
print("vect_b_x = \n", vect_b_x)

mat_p = 
 [[ 1  0]
 [-1  1]]
inv_mat_p = 
 [[1. 0.]
 [1. 1.]]
vect_b_x = 
 [ 0 30]


The following expression must be positive:
$$
d_\mathrm{min}-[G]\cdot[P]^{-1}\mathbf{b}_X
$$

In [6]:
d_min - np.dot(mat_g @ inv_mat_p, vect_b_x)

array([0.])

In [7]:
np.dot(inv_mat_p, np.array([0, 0] )) + vect_b_x

array([ 0., 30.])

In [8]:
mat_p @ np.array([0, d_min]) - vect_b_x

array([0, 0])

## Solving with Simplex algorithm

In [9]:
x = cp.Variable(shape=(2,), name="X")
constraints = [
    cp.matmul(mat_a @ inv_mat_p, x) == -np.dot(mat_a @ inv_mat_p, vect_b_x), 
    cp.matmul(mat_g @ inv_mat_p, x) >= 0 #-mat_g @ inv_mat_p @ vect_b + d_min * np.ones(shape=(4,))
]
objective = cp.Minimize(cp.matmul(lform_c @ inv_mat_p, x))
problem = cp.Problem(objective, constraints)
solution = problem.solve()

In [10]:
with np.printoptions(suppress=True, precision=6):
    print("Solution in y-coordinates = \n", inv_mat_p @ (x.value + vect_b_x))

Solution in y-coordinates = 
 [-15.  15.]
